In [67]:
import json
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV


In [8]:
#change to your own directory
%cd ~/Desktop/Uni_Zurich/2019HS/Machine_Learning/Exercises/Exercise_1

/Users/debora/Desktop/Uni_Zurich/2019HS/Machine_Learning/Exercises/Exercise_1


In [22]:
#process the first file
filename1 = "tweets.json"
infile1 = open(filename1, 'r')

In [23]:
import json
import pandas as pd
train_tweets = []
for line in infile1: #put each line in a list of lines
    j_content = json.loads(line)
    train_tweets.append(j_content)
tweets = pd.DataFrame(train_tweets, columns=['ID', 'Tweet']) #make a dataframe out of it


In [24]:
tweets #looks good!

,ID,Tweet
0,483885347374243841,اللهم أفرح قلبي وقلب من أحب وأغسل أحزاننا وهمو...
1,484023414781263872,إضغط على منطقتك يتبين لك كم يتبقى من الوقت عن ...
2,484026168300273664,اللَّهٌمَّ صَلِّ وَسَلِّمْ عَلىٰ نَبِيِّنَآ مُ...
3,483819942878650369,@Dinaa_ElAraby اها يا بيبي والله اتهرست علي تو...
4,483793769079123971,• افضل كتاب قرأته هو : أمي (ابراهام لنكولن)\n🌹...
5,483934868070350849,@hudc7721 انتظري اجل \nخيره لك يارب 😘
6,483863369972473856,(وإن تجهر بالقول فإنه يعلم السر وأخفى) [طه:7]\...
7,483871567311413248,ﺧﻟك ﻋزﯾز آﻟﻧﻓس ﻟۈ ﮪﻣۈﻣك ﺟﺑآللاﭠﺷﺷﮐي ﻟﻟﻧآﺳس ﻣن ...
8,483931429902884864,عشان الجنّة أجمل ؟ الله يبعدنا عن كل ذنب مايخ...
9,483773756897124352,توجيه كيفية تثبيت البرامج الثابتة ROM التحميل ...


In [36]:
#deal with both label documents
train_label1 = pd.read_csv("labels-train+dev.tsv",sep='\t', header=None, names=['Label', 'ID'])
train_label2 = pd.read_csv("labels-test.tsv", sep='\t', header=None, names=['Label', 'ID'])

In [37]:
train_label1 #looks about right

,Label,ID
0,ar,483762194908479488
1,ar,483762916097654784
2,ar,483764828784582656
3,ar,483765526683209728
4,ar,483768342315282432
5,ar,483770765985083392
6,ar,483770900127285248
7,ar,483770997892345857
8,ar,483773690769702912
9,ar,483773756897124352


In [40]:
labels = pd.concat([train_label1, train_label2]) #join both together

In [41]:
labels

,Label,ID
0,ar,483762194908479488
1,ar,483762916097654784
2,ar,483764828784582656
3,ar,483765526683209728
4,ar,483768342315282432
5,ar,483770765985083392
6,ar,483770900127285248
7,ar,483770997892345857
8,ar,483773690769702912
9,ar,483773756897124352


In [47]:
tweets['ID']=tweets['ID'].astype(int) #to allow for merge, need the same type
data = pd.merge(tweets, labels, on='ID') #merge by ID

In [48]:
data

,ID,Tweet,Label
0,483885347374243841,اللهم أفرح قلبي وقلب من أحب وأغسل أحزاننا وهمو...,ar
1,484023414781263872,إضغط على منطقتك يتبين لك كم يتبقى من الوقت عن ...,ar
2,484026168300273664,اللَّهٌمَّ صَلِّ وَسَلِّمْ عَلىٰ نَبِيِّنَآ مُ...,ar
3,483819942878650369,@Dinaa_ElAraby اها يا بيبي والله اتهرست علي تو...,ar
4,483793769079123971,• افضل كتاب قرأته هو : أمي (ابراهام لنكولن)\n🌹...,ar
5,483934868070350849,@hudc7721 انتظري اجل \nخيره لك يارب 😘,ar
6,483863369972473856,(وإن تجهر بالقول فإنه يعلم السر وأخفى) [طه:7]\...,ar
7,483871567311413248,ﺧﻟك ﻋزﯾز آﻟﻧﻓس ﻟۈ ﮪﻣۈﻣك ﺟﺑآللاﭠﺷﺷﮐي ﻟﻟﻧآﺳس ﻣن ...,ar
8,483931429902884864,عشان الجنّة أجمل ؟ الله يبعدنا عن كل ذنب مايخ...,ar
9,483773756897124352,توجيه كيفية تثبيت البرامج الثابتة ROM التحميل ...,ar


In [147]:
data_no_na = data.dropna() #drop the NaN, apparently not necessary
data_copy = data_no_na.copy() #make a copy of the data
data_copy = data_copy.sample(frac=1).reset_index(drop=True) #shuffle the data
train_set = data_copy.sample(frac=0.9, random_state=0) #take 90% of the data, reshuffle
test_set = data_copy.drop(train_set.index) #take 10% that remain

66921
66921


In [121]:
print(train_set.head()) #some checks
print(type(train_set))

                       ID                                              Tweet  \
35801  495281803813330944  I just signed up to win some great prizes from...   
44888  486402557820928000         niggas be sick when they see you moved on.   
8125   494376293887377408  when someone subtweets...lol https://t.co/b20v...   
52485  485008158608203778  @naochisa 予定よりはやかった？？？よーやくやね！！！妹さんおめでとうーー♪ヽ(´▽｀)/   
50266  494896773220286464  @sonriemeamors jajaja hay si hay si y tu ami? ...   

      Label  
35801    en  
44888    en  
8125     en  
52485    ja  
50266    es  
<class 'pandas.core.frame.DataFrame'>


In [122]:
train = train_set.drop('ID', axis=1) #drop the ID, not needed anymore
test = test_set.drop('ID', axis=1)

In [123]:
train.head()

,Tweet,Label
35801,I just signed up to win some great prizes from...,en
44888,niggas be sick when they see you moved on.,en
8125,when someone subtweets...lol https://t.co/b20v...,en
52485,@naochisa 予定よりはやかった？？？よーやくやね！！！妹さんおめでとうーー♪ヽ(´▽｀)/,ja
50266,@sonriemeamors jajaja hay si hay si y tu ami? ...,es


In [124]:
test.head()

,Tweet,Label
10,ならんならんよお客様,ja
11,Pagi~,id
13,昨日録った水トクも見たけど 合成ばっかで全然怖くなかったしw\n何でいつも髪の長い白い服着た...,ja
19,@Nati5538 @MartinNicolasL2 jajaja buenoooo:),es
26,"Como Trabajar Legalmente En Canadá, Secreto Le...",es


In [60]:
test

,Tweet,Label
10,はぁ、どうしよw\nかわいそうww,ja
11,#MTVHottest One Direction Harry Styles,en
13,Uh oh well.,en
19,@Ashton5SOS just ur normal target boy,en
26,trip nila ako kanina :--),tl
30,シュ何ム,ja
41,Ma quando hanno dato la notizia del matrimonio...,it
43,I come to realize most females i know can't li...,en
52,@omaketuki3 現実だったら良かったのに,ja
55,"Today stats: One follower, No unfollowers and ...",en


In [125]:
train.describe() #descrption of data

,Tweet,Label
count,60229,60229
unique,60132,79
top,:(,en
freq,9,21246


In [126]:
train.groupby('Label').size() #more description of teh data. See that there are lots of Arabian and English tweets, also quite a few in Spanish and Portuguese

Label
ar          2536
ar_LATN       13
az             3
bg             4
bn             8
bs             3
ca            23
cs             5
cy             1
da             7
de           203
dv             1
el            48
en         21246
es          6720
et             2
eu             2
fa            23
fi            22
fr          1050
gl             3
ha             1
he            38
hi            17
hi-Latn       15
hr             6
ht             3
hu            14
hy             2
id          3500
           ...  
pl           107
ps             1
ps_LATN        1
pt          3256
ro            11
ru          1095
si             1
sk             1
sl             2
sq             7
sr            25
su             8
sv            67
sw             7
ta             9
ta_LATN        1
th           493
tl           363
tn             1
tr           754
uk            16
und         5462
ur            10
ur_LATN       12
vi            21
xh             2
yo             1
zh-CN   

In [127]:
X_train = train.Tweet #split the data in Series
y_train = train.Label
X_test = test.Tweet
y_test = test.Label

In [128]:
print(type(X_train))
print(type(y_train))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [129]:
from sklearn.preprocessing import LabelEncoder(
#encode the labels. First step means simple encoding, the second makes a series out of the array that was outputted and
    #the third step means we output strings again (strings are apparently needed as a format)

label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train)
y_train_series = pd.Series(y_train_encoded)
y_train_str = y_train_series.apply(str)

y_test_encoded = label_encoder.fit_transform(y_test)
y_test_series = pd.Series(y_test_encoded)
y_test_str = y_test_series.apply(str)

In [210]:
pipeline1 = Pipeline([('tfidf', TfidfVectorizer()), ('clf0', MultinomialNB())]) #first test with a pipeline

In [211]:
pipeline1.fit(X_train, y_train_str) #using y_train_str

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...ue,
        vocabulary=None)), ('clf0', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [213]:
pipeline1.predict(X_test)

array(['13', '29', '13', ..., '13', '32', '13'], dtype='<U2')

In [215]:
y_test_str #doesn't look too right, but it is a very simple model

0       18
1       16
2       18
3        9
4        9
5        8
6       42
7        8
8       18
9        8
10      42
11      18
12       8
13       8
14      30
15       8
16       8
17       8
18      18
19      30
20      18
21       9
22      42
23       8
24       0
25      42
26       9
27      28
28      28
29      18
        ..
6662     8
6663    18
6664     0
6665    18
6666     9
6667    18
6668     8
6669    42
6670    18
6671     8
6672     8
6673     8
6674     8
6675     9
6676    42
6677    16
6678    18
6679    20
6680     8
6681     8
6682     8
6683     8
6684    30
6685     8
6686     8
6687     8
6688    20
6689    18
6690    18
6691     8
Length: 6692, dtype: object

In [134]:
#honestly i don't know why i did that anymore!!!
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_tranformer = TfidfTransformer(smooth_idf=True).fit(X_train_counts)
X_train_tfidf = tfidf_tranformer.transform(X_train_counts)
print(type(X_train_tfidf))

<class 'scipy.sparse.csr.csr_matrix'>


In [135]:
#same here
nb_clf = MultinomialNB()
nb_clf.fit(X_train_tfidf, y_train_str)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [137]:
#follows here
nb_clf.predict(X_train_tfidf)

array(['13', '13', '13', ..., '29', '29', '13'], dtype='<U2')

In [138]:
#and here
y_train_str

0        13
1        13
2        13
3        32
4        14
5        13
6        13
7        65
8        14
9        13
10        0
11       13
12       14
13       32
14       13
15       13
16       32
17       13
18       29
19       13
20       13
21       13
22       14
23       13
24       52
25       13
26       68
27       13
28       14
29       13
         ..
60199    32
60200    13
60201    32
60202    29
60203    14
60204    13
60205    32
60206    13
60207    13
60208    32
60209    13
60210    32
60211    29
60212    14
60213    13
60214    32
60215    70
60216    19
60217    13
60218    13
60219    54
60220    32
60221    32
60222    32
60223    13
60224    32
60225    70
60226    29
60227    29
60228    13
Length: 60229, dtype: object

In [177]:
#first serious pipeline with ngrams and tfidf
pipeline_NB01 = Pipeline([
    ('ngram', CountVectorizer(ngram_range=(1, 4), analyzer='word')),
    ('tfidf', TfidfTransformer()),
    ('clf01', MultinomialNB())
])

In [178]:
#param_grid is fitting the pipeline_NB01
param_grid01 = {'clf01__alpha': [0.2, 0.6, 0.8, 1.0],
                 'clf01__fit_prior': [True, False]}  #'ngram__ngram_range': [(1, 1), (1, 2), (1, 4)]


In [180]:
#This model here seems to work
gs_NB01= GridSearchCV(pipeline_NB01, param_grid01, cv=2, n_jobs=2, verbose=1)
gs_NB01.fit(X_train, y_train)

Fitting 2 folds for each of 8 candidates, totalling 16 fits


/Users/debora/Envs/nlppython/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=2.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  16 out of  16 | elapsed:  1.9min finished


GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('ngram', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 4), preprocessor=None, stop_words=None,
        stri...ear_tf=False, use_idf=True)), ('clf01', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params=None, iid='warn', n_jobs=2,
       param_grid={'clf01__alpha': [0.2, 0.6, 0.8, 1.0], 'clf01__fit_prior': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [171]:
gs_NB01.predict(X_test)

array(['ar', 'id', 'ar', ..., 'ar', 'ja', 'en'], dtype='<U7')

In [172]:
y_test #results are quite terrible though

10        ja
11        id
13        ja
19        es
26        es
30        en
41       und
43        en
52        ja
55        en
73       und
75        ja
77        en
93        en
95        ru
100       en
102       en
104       en
106       ja
107       ru
108       ja
110       es
120      und
128       en
131       ar
163      und
171       es
177       pt
195       pt
197       ja
        ... 
66636     en
66639     ja
66642     ar
66669     ja
66672     es
66674     ja
66682     en
66703    und
66706     ja
66711     en
66714     en
66725     en
66729     en
66730     es
66752    und
66755     id
66760     ja
66768     ko
66781     en
66789     en
66795     en
66824     en
66832     ru
66833     en
66856     en
66862     en
66875     ko
66896     ja
66901     ja
66902     en
Name: Label, Length: 6692, dtype: object

But here we really need to find the best_model out of this data i got. I don't remember how to see it in the matrix, see Tutorial!

In [216]:
#Average word length extractor, inspired  by https://michelleful.github.io/code-blog/2015/06/20/pipelines/)
class AverageWordLengthExtractor(BaseEstimator, TransformerMixin):
    """Takes in dataframe, extracts tweet column, outputs average word length"""

    def __init__(self):
        pass

    def average_word_length(self, tweet):
        """Helper code to compute average word length of a tweet"""
        return np.mean([len(word) for word in tweet.split()])

    def transform(self, df, y=None):
        """The workhorse of this feature extractor"""
        #return map(self.average_word_length, series)
        return df.apply(self.average_word_length)

    def fit(self, df, y=None):
        """Returns `self` unless something different happens in train and test"""
        return self

In [217]:
pipeline_NB1 = Pipeline([
    ('feats', FeatureUnion([
        ('ngram', CountVectorizer(ngram_range=(1, 4), analyzer='word')),
        ('tfidf', TfidfTransformer()),
        ('ave', AverageWordLengthExtractor())
    ])),
    ('nb_clf', MultinomialNB()) # classifier
])

In [218]:
#-*- coding: utf-8 -*-

In [219]:
param_grid1 = {'nb_clf__alpha': [0.2, 0.6, 0.8, 1.0],
                 'nb_clf__fit_prior': [True, False]}  #'ngram__ngram_range': [(1, 1), (1, 2), (1, 4)]


In [220]:
gs_NB1= GridSearchCV(pipeline_NB1, param_grid1, cv=2, n_jobs=2, verbose=1)
gs_NB1.fit(X_train, y_train)

Fitting 2 folds for each of 8 candidates, totalling 16 fits


/Users/debora/Envs/nlppython/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=2.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


ValueError: could not convert string to float: 'ញាំថ្នាំផ្តាសសាយខ្លាំពេកឡើងគេញលែងចង់ចង់ក្រោកហើយ....'

In [ ]:
#have a try with SGD, same features!
pipeline_SGD = Pipeline([
    ('feats', FeatureUnion([
        ('ngram', CountVectorizer(ngram_range=(1, 4), analyzer='word')),
        ('tfidf', TfidfTransformer()), # can pass in either a pipeline
        ('ave', AverageWordLengthExtractor()), # or a transformer
    ])),
    ('SGD_clf', SGDClassifier())# classifier
])

In [209]:
grid_param_SGD = {'SGD_clf__loss': ['hinge', 'log', 'modified huber'],
                  'SGD_clf__penalty': ['none', 'l1', 'l2'],
                  'SGD_clf__max_iter': [50, 100, 500, 1000]}

In [208]:
gs_SGD = GridSearchCV(pipeline_SGD, grid_param_SGD, cv=2, n_jobs=4, verbose=1)
gs_SGD.fit(X_train, y_train)
##NOTE: it crashed^s at the same point: could not convert string to float: 'ញាំថ្នាំផ្តាសសាយខ្លាំពេកឡើងគេញលែងចង់ចង់ក្រោកហើយ....'
#Maybe some problem with the encoding?

Fitting 2 folds for each of 36 candidates, totalling 72 fits


/Users/debora/Envs/nlppython/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=2.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


ValueError: could not convert string to float: 'ញាំថ្នាំផ្តាសសាយខ្លាំពេកឡើងគេញលែងចង់ចង់ក្រោកហើយ....'